In [18]:
import spacy
import glob
import json
import random
from tqdm import tqdm, tqdm_notebook
from spacy.util import minibatch, compounding
from sklearn.utils import shuffle
from pathlib import Path

In [26]:
spacy.require_gpu()

True

### Load data

In [3]:
TRAIN_DATA = []

In [4]:
files = glob.glob("train/*.json")

In [5]:
for f in files:
    with open(f) as fl:
        js = json.load(fl)
        for j in js:
            TRAIN_DATA.append(tuple(j))

## Existing model 

In [23]:
# from thinc.neural.ops import CupyOps
# spacy.require_gpu()
# nlp = spacy.load("en")
# n_iter=1
# if 'ner' not in nlp.pipe_names:
#     ner = nlp.create_pipe('ner')
#     nlp.add_pipe(ner)
# # otherwise, get it, so we can add labels to it
# else:
#     ner = nlp.get_pipe('ner')

# ner.add_label("DISEASE")
# ner.add_label("GENE")

# optimizer = nlp.entity.create_optimizer()

# other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
# with nlp.disable_pipes(*other_pipes):  # only train NER
#     for itn in range(n_iter):
#         random.shuffle(TRAIN_DATA)
#         losses = {}
#         # batch up the examples using spaCy's minibatch
#         batches = minibatch(TRAIN_DATA, size=512)
#         for batch in batches:
#             texts, annotations = zip(*batch)
#             nlp.update(texts, annotations, sgd=optimizer, drop=0.35,
#                        losses=losses)
#         print('Losses', losses)

### New model

In [21]:
output_dir="models/"
n_iter=2

In [7]:
# load model 
spacy.util.use_gpu(0)
# nlp = spacy.load('en')
nlp = spacy.blank('en')

In [8]:
# ner = nlp.get_pipe("ner")

ner = nlp.create_pipe('ner')
nlp.add_pipe(ner)

In [9]:
# for _, annotations in TRAIN_DATA:
#     for ent in annotations.get('entities'):
#         ner.add_label(ent[2])
ner.add_label("DISEASE")
ner.add_label("GENE")

In [10]:
optimizer = nlp.begin_training(use_gpu=True, device=0)

In [11]:
batch_size = 512

In [12]:
batch_size = 512
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
with nlp.disable_pipes(*other_pipes):  # only train NER
    # reset and initialize the weights randomly – but only if we're
    # training a new model
    for itn in range(n_iter):
        print("Epoch : ", itn, " batch : ", len(TRAIN_DATA)/batch_size)
        random.shuffle(TRAIN_DATA)
        losses = {}
        # batch up the examples using spaCy's minibatch
        batches = minibatch(TRAIN_DATA, size=batch_size)
        for batch in tqdm(batches):
            texts, annotations = zip(*batch)
            nlp.update(
                texts,  # batch of texts
                annotations,  # batch of annotations
                drop=0.5,  # dropout - make it harder to memorise data
                losses=losses,sgd=optimizer
            )
        print("Losses", losses)
        output_dir = Path(output_dir+"fromscratch/epoch_"+str(itn))
        if not output_dir.exists():
            output_dir.mkdir()
        
        nlp.meta['name'] = "ner_fromscratch"
        nlp.to_disk(output_dir)

0it [00:00, ?it/s]

('Epoch : ', 0, ' batch : ', 861)
('Losses', {u'ner': 0.034703700337558985})


In [13]:
# for text, _ in shuffle(TRAIN_DATA)[0:10]:
#     doc = nlp(text)
#     print("-------------------")
#     print(text)
#     print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
#     print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

In [14]:
nlp.meta['name'] = "ner_fromscratch" 

In [16]:
nlp.to_disk("models")

In [30]:
"""Example of training an additional entity type

This script shows how to add a new entity type to an existing pre-trained NER
model. To keep the example short and simple, only four sentences are provided
as examples. In practice, you'll need many more — a few hundred would be a
good start. You will also likely need to mix in examples of other entity
types, which might be obtained by running the entity recognizer over unlabelled
sentences, and adding their annotations to the training set.

The actual training is performed by looping over the examples, and calling
`nlp.entity.update()`. The `update()` method steps through the words of the
input. At each word, it makes a prediction. It then consults the annotations
provided on the GoldParse instance, to see whether it was right. If it was
wrong, it adjusts its weights so that the correct action will score higher
next time.

After training your model, you can save it to a directory. We recommend
wrapping models as Python packages, for ease of deployment.

For more details, see the documentation:
* Training: https://spacy.io/usage/training
* NER: https://spacy.io/usage/linguistic-features#named-entities

Compatible with: spaCy v2.0.0+
"""
from __future__ import unicode_literals, print_function

import plac
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding


# new entity label
LABEL = 'ANIMAL'

# training data
# Note: If you're using an existing model, make sure to mix in examples of
# other entity types that spaCy correctly recognized before. Otherwise, your
# model might learn the new type, but "forget" what it previously knew.
# https://explosion.ai/blog/pseudo-rehearsal-catastrophic-forgetting
TRAIN_DATA = [
    ("Horses are too tall and they pretend to care about your feelings", {
        'entities': [(0, 6, 'ANIMAL')]
    }),

    ("Do they bite?", {
        'entities': []
    }),

    ("horses are too tall and they pretend to care about your feelings", {
        'entities': [(0, 6, 'ANIMAL')]
    }),

    ("horses pretend to care about your feelings", {
        'entities': [(0, 6, 'ANIMAL')]
    }),

    ("they pretend to care about your feelings, those horses", {
        'entities': [(48, 54, 'ANIMAL')]
    }),

    ("horses?", {
        'entities': [(0, 6, 'ANIMAL')]
    })
]


@plac.annotations(
    model=("Model name. Defaults to blank 'en' model.", "option", "m", str),
    new_model_name=("New model name for model meta.", "option", "nm", str),
    output_dir=("Optional output directory", "option", "o", Path),
    n_iter=("Number of training iterations", "option", "n", int))
def main(model=None, new_model_name='animal', output_dir=None, n_iter=10):
    """Set up the pipeline and entity recognizer, and train the new entity."""
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank('en')  # create blank Language class
        print("Created blank 'en' model")
    # Add entity recognizer to model if it's not in the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner)
    # otherwise, get it, so we can add labels to it
    else:
        ner = nlp.get_pipe('ner')

    ner.add_label(LABEL)   # add new entity label to entity recognizer
    if model is None:
        optimizer = nlp.begin_training()
    else:
        # Note that 'begin_training' initializes the models, so it'll zero out
        # existing entity types.
        optimizer = nlp.entity.create_optimizer()

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            losses = {}
            # batch up the examples using spaCy's minibatch
            batches = minibatch(TRAIN_DATA, size=compounding(4., 32., 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(texts, annotations, sgd=optimizer, drop=0.35,
                           losses=losses)
            print('Losses', losses)

    # test the trained model
    test_text = 'Do you like horses?'
    doc = nlp(test_text)
    print("Entities in '%s'" % test_text)
    for ent in doc.ents:
        print(ent.label_, ent.text)

    # save model to output directory
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.meta['name'] = new_model_name  # rename model
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

        # test the saved model
        print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        doc2 = nlp2(test_text)
        for ent in doc2.ents:
            print(ent.label_, ent.text)

main("en")

Loaded model 'en'


KeyError: <type 'numpy.object_'>